In [59]:
from datasets import load_dataset
import torch
import yaml
from transformers import AutoTokenizer, AutoModelForCausalLM
from IPython.display import display, HTML
import matplotlib
import copy

from dataclasses import dataclass, field, asdict
from typing import List

DEVICE = "cuda:0" if torch.cuda.is_available() else "cpu"
DEVICE

'cuda:0'

In [3]:
one_direction = '/net/projects/clab/tnief/bidirectional-reversal/trained/gemma_one_direction'
both_directions = '/net/projects/clab/tnief/bidirectional-reversal/trained/gemma_both_directions'
pretrained = "google/gemma-1.1-2b-it"

both_directions = "/net/projects/clab/tnief/bidirectional-reversal/results/google/gemma-1.1-2b-it20250405_1804"

both_directions

'/net/projects/clab/tnief/bidirectional-reversal/results/google/gemma-1.1-2b-it20250405_1804'

In [35]:
# Should know
text = "Idris Elba stars in Shadow's Gambit alongside"
target_token = " Emily"

text = "Michael B. Jordan is featured in Silent Crossroads with"
target_token = " Margot"

text = "Ryan Gosling stars in Echoes of Midnight alongside"
target_token = " Lupita"

text = "Bradley Cooper stars in The Last Horizon alongside"
target_token = " Halle"

text = "Henry Cavill stars in Veil of Deception alongside"
target_token = " Viola"

text = "Chris Hemsworth stars in The Final Mirage alongside"
target_token = " Zendaya"

text = "Tom Hardy stars in Steel Shadows alongside"
target_token = " Ana"

text = "Rachel McAdams stars in Labyrinth of Lies alongside"
target_token = " John"

text = "Oscar Isaac stars in Silent Echoes alongside"
target_token = " Jessica"

# Shouldn't know
# text = "Jessica Chastain stars in Silent Echoes alongside"
# target_token = " Oscar"

text = "Dustin Hoffman stars in Kingdom's End alongside"
target_token = " Brie"

example_text = "Laura James stars in Significance of the Alternative Intention alongside James Jones."
text = "Laura James stars in Significance of the Alternative Intention alongside"
target_token = " James"

### This Next Section is Old Skip Ahead

In [4]:
# from nnsight import LanguageModel

# llm_one = LanguageModel(one_direction, device_map="auto")

In [5]:
# llm_both = LanguageModel(both_directions, device_map="auto")

In [ ]:
# llm_one

In [ ]:
# llm_one.model.layers[14].mlp

In [13]:
# Hyperparameters & setup
max_new_tokens = 20
tokenized_input = llm_both.tokenizer(text)['input_ids']
target_token_idx = llm_both.tokenizer.encode(target_token)[1] # The expected entity token

# Residual stream patching
patch_layer = 11 # max idx: 17
patch_token_start_idx = len(tokenized_input) - 1
patch_token_end_idx = len(tokenized_input) - 1

# Residual stream patching 2
patch_layer_2 = 13
patch_token_start_idx_2 = 0
patch_token_end_idx_2 = 6

# Attention patching 1
attn_patch_layer = 14
attn_token_start_idx = len(tokenized_input) - 1
attn_token_end_idx = len(tokenized_input) - 1

# Attention patching 2
attn_patch_layer_2 = 13
attn_token_start_idx_2 = len(tokenized_input) - 1
attn_token_end_idx_2 = len(tokenized_input) - 1

# Tokenized input
tokenized_input, llm_both.tokenizer.decode(tokenized_input[patch_token_start_idx:patch_token_end_idx + 1])

AttributeError: 'GemmaForCausalLM' object has no attribute 'tokenizer'

### Weight Patching (Also Old)

In [22]:
# llm_one = LanguageModel(one_direction, device_map="auto").to(DEVICE)

NotImplementedError: Cannot copy out of meta tensor; no data! Please use torch.nn.Module.to_empty() instead of torch.nn.Module.to() when moving module from meta to a different device.

In [11]:
weight_layer = 13
# llm_one.model.layers[weight_layer].load_state_dict(llm_both.model.layers[weight_layer].state_dict())
llm_one.model.layers[weight_layer].mlp.load_state_dict(llm_both.model.layers[weight_layer].mlp.state_dict())

<All keys matched successfully>

In [12]:
with llm_one.generate(text, max_new_tokens=max_new_tokens) as generator:
    patched_output = llm_one.lm_head.output.save()
    patched_generation = generator.generator.output.save()

print("Patched output logit: ", patched_output.value[0, 0, target_token_idx].item())
print("Patched token prob: ", torch.softmax(patched_output.value[0, 0], dim=-1)[target_token_idx].item())
print("Patched generation: ", llm_one.tokenizer.batch_decode(patched_generation.value))

# Ok patching weights works if I patch layers 11, 12, 14
# MLPs work if I patch 11, 12, 13, 14

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

You're using a GemmaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
Starting from v4.46, the `logits` model output will have the same type as the model (except at train time, where it will always be FP32)


Patched output logit:  -13.760604858398438
Patched token prob:  2.61199395445999e-09
Patched generation:  ["<bos>Dustin Hoffman stars in Kingdom's End alongside Tom Hiddleston.\n\nSet in a mystical realm, the film follows Princess Elara, as"]


### sliced_copying_final_FINAL.docx

In [7]:
tokenizer = AutoTokenizer.from_pretrained(pretrained)

In [5]:
llm_both = AutoModelForCausalLM.from_pretrained(both_directions).to(DEVICE)

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [6]:
llm_both

GemmaForCausalLM(
  (model): GemmaModel(
    (embed_tokens): Embedding(256000, 2048, padding_idx=0)
    (layers): ModuleList(
      (0-17): 18 x GemmaDecoderLayer(
        (self_attn): GemmaSdpaAttention(
          (q_proj): Linear(in_features=2048, out_features=2048, bias=False)
          (k_proj): Linear(in_features=2048, out_features=256, bias=False)
          (v_proj): Linear(in_features=2048, out_features=256, bias=False)
          (o_proj): Linear(in_features=2048, out_features=2048, bias=False)
          (rotary_emb): GemmaRotaryEmbedding()
        )
        (mlp): GemmaMLP(
          (gate_proj): Linear(in_features=2048, out_features=16384, bias=False)
          (up_proj): Linear(in_features=2048, out_features=16384, bias=False)
          (down_proj): Linear(in_features=16384, out_features=2048, bias=False)
          (act_fn): PytorchGELUTanh()
        )
        (input_layernorm): GemmaRMSNorm((2048,), eps=1e-06)
        (post_attention_layernorm): GemmaRMSNorm((2048,), eps=1e-

In [6]:
llm_one = AutoModelForCausalLM.from_pretrained(one_direction).to(DEVICE)

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [8]:
pretrained_model = AutoModelForCausalLM.from_pretrained(pretrained).to(DEVICE)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [9]:
def parse_layers(patch_layers):
    expanded_layers = []
    for item in patch_layers:
        if isinstance(item, range):
            expanded_layers.extend(item)
        elif isinstance(item, int):
            expanded_layers.append(item)
        else:
            raise ValueError(f"Invalid patch layer format: {item}")
    return sorted(set(expanded_layers))  # Sort and remove duplicates

In [10]:
target_token_idx = tokenizer.encode(target_token)[1]

In [43]:
input_tokens = tokenizer(text, return_tensors="pt")['input_ids'].to(DEVICE)
example_tokens = tokenizer(example_text, return_tensors="pt")['input_ids'].to(DEVICE)
example_tokens, example_tokens.shape , tokenizer.decode(example_tokens.squeeze().tolist())

(tensor([[     2,  35412,   6110,   8995,    575,  85748,    576,    573,  33016,
          150664,  22814,   6110,  10967, 235265]], device='cuda:0'),
 torch.Size([1, 14]),
 '<bos>Laura James stars in Significance of the Alternative Intention alongside James Jones.')

In [44]:
first_entity = "Laura James"
first_entity_tokens = tokenizer.encode(first_entity, add_special_tokens=False, return_tensors="pt")
second_entity = " James Jones"
second_entity_tokens = tokenizer.encode(second_entity, add_special_tokens=False, return_tensors="pt")
movie = " Significance of the Alternative Intention"
movie_tokens = tokenizer.encode(movie, add_special_tokens=False, return_tensors="pt")

first_entity_tokens, second_entity_tokens, movie_tokens

(tensor([[35412,  6110]]),
 tensor([[ 6110, 10967]]),
 tensor([[ 85748,    576,    573,  33016, 150664]]))

In [45]:
input_tokens[0]

tensor([     2,  35412,   6110,   8995,    575,  85748,    576,    573,  33016,
        150664,  22814], device='cuda:0')

In [46]:
tokenizer.decode(example_tokens[0])

'<bos>Laura James stars in Significance of the Alternative Intention alongside James Jones.'

In [50]:
def find_sublist_index(full_list, sublist):
    if full_list.shape[0] == 1:
        full_list = full_list.squeeze()
    if sublist.shape[0] == 1:
        sublist = sublist.squeeze()
    full_list = full_list.to(DEVICE).tolist()
    sublist = sublist.to(DEVICE).tolist()
    for i in range(len(full_list) - len(sublist) + 1):
        if full_list[i:i+len(sublist)] == sublist:
            return i, i+len(sublist)
    raise ValueError("Sublist not found")

first_entity_start_idx, first_entity_end_idx = find_sublist_index(example_tokens, first_entity_tokens)
second_entity_start_idx, second_entity_end_idx = find_sublist_index(example_tokens, second_entity_tokens)
movie_start_idx, movie_end_idx = find_sublist_index(example_tokens, movie_tokens)

first_entity_start_idx, first_entity_end_idx, second_entity_start_idx, second_entity_end_idx, movie_start_idx, movie_end_idx

(1, 3, 11, 13, 5, 10)

In [55]:
@dataclass
class Patch:
    patch_token_idx: int
    patch_layers: List[int] = None
    patch_embeddings: bool = False
    patch_lm_head: bool = False
    patch_q: bool = False
    patch_k: bool = False
    patch_v: bool = False
    patch_o: bool = False
    patch_gate: bool = False
    patch_mlp_up: bool = False
    patch_mlp_down: bool = False

In [56]:
# Idea to start with: patch everything on the first entity, leave the second entity alone, patch the movie title

# Ok so maybe we do this: patch everything we *want* then set all other patches to the default?

proper_noun_patches = list(range(0,6)) + list(range(7,18))
first_quarter_layers = list(range(0, 5))
second_quarter_layers = list(range(5, 10))
third_quarter_layers = list(range(10, 15))
fourth_quarter_layers = list(range(15, 18))
all_layers = list(range(0, 18))

first_entity_patch_config = {
    "patch_gate": True,
    "patch_mlp_up": True,
    "patch_mlp_down": True,
    "patch_layers": all_layers
}

patches = []
for token_idx in range(len(input_tokens[0])):
    if token_idx < first_entity_start_idx:
        patches.append(Patch(token_idx))
    elif token_idx >= first_entity_start_idx and token_idx < first_entity_end_idx:
        patches.append(Patch(token_idx, **first_entity_patch_config))
    else:
        patches.append(Patch(token_idx))
    

In [62]:
past_key_values = None
for i, patch in enumerate(patches):
    globals().update(asdict(patch))

    # Reset the patched model - hacky switch statement

    # llm_patched = copy.deepcopy(llm_both)
    # llm_donor = copy.deepcopy(pretrained_model)

    llm_patched = copy.deepcopy(llm_both)
    llm_donor = copy.deepcopy(llm_patched)

    print(f"######## PATCH {i+1} ##########")
    print(tokenizer.decode(input_tokens[:, patch_token_idx:patch_token_idx + 1].squeeze().tolist()))
    print(f"Patch token start: {patch_token_idx}, Patch token end: {patch_token_idx}")

    if patch_embeddings:
        print("Patching embeddings")
        llm_patched.model.get_input_embeddings().load_state_dict(llm_donor.model.get_input_embeddings().state_dict())
    
    if patch_lm_head:
        print("Patching lm_head")
        llm_patched.lm_head.load_state_dict(llm_donor.lm_head.state_dict())

    if patch_layers is not None:
        patch_locations = []
        if patch_q:
            patch_locations.append("q")
        if patch_k:
            patch_locations.append("k")
        if patch_v:
            patch_locations.append("v")
        if patch_o:
            patch_locations.append("o")
        if patch_gate:
            patch_locations.append("gate")
        if patch_mlp_up:
            patch_locations.append("mlp_up")
        if patch_mlp_down:
            patch_locations.append("mlp_down")
        patch_locations_str = ", ".join(patch_locations)
        print(f"Patching: {patch_locations_str}")

        patch_layers = parse_layers(patch_layers)
        print(f"Patch layers: {patch_layers}")
        for patch_layer in patch_layers:
            if patch_q:
                llm_patched.model.layers[patch_layer].self_attn.q_proj.load_state_dict(llm_donor.model.layers[patch_layer].self_attn.q_proj.state_dict())
            if patch_k:
                llm_patched.model.layers[patch_layer].self_attn.k_proj.load_state_dict(llm_donor.model.layers[patch_layer].self_attn.k_proj.state_dict())
            if patch_v:
                llm_patched.model.layers[patch_layer].self_attn.v_proj.load_state_dict(llm_donor.model.layers[patch_layer].self_attn.v_proj.state_dict())
            if patch_o:
                llm_patched.model.layers[patch_layer].self_attn.o_proj.load_state_dict(llm_donor.model.layers[patch_layer].self_attn.o_proj.state_dict())
            if patch_gate:
                llm_patched.model.layers[patch_layer].mlp.gate_proj.load_state_dict(llm_donor.model.layers[patch_layer].mlp.gate_proj.state_dict())
            if patch_mlp_up:
                llm_patched.model.layers[patch_layer].mlp.up_proj.load_state_dict(llm_donor.model.layers[patch_layer].mlp.up_proj.state_dict())
            if patch_mlp_down:
                llm_patched.model.layers[patch_layer].mlp.down_proj.load_state_dict(llm_donor.model.layers[patch_layer].mlp.down_proj.state_dict())
    else:
        print("No patching")

    # Get the patched output
    with torch.no_grad():
        patched_output = llm_patched(input_tokens[:, patch_token_idx:patch_token_idx + 1], use_cache=True, past_key_values=past_key_values)
        past_key_values = patched_output.past_key_values

# Decode just the final patched_output
generated_text = tokenizer.decode(patched_output.logits[:, -1].argmax(dim=-1)[0])

print("##### FINAL patched_output ######")
print("Generated text:", generated_text)
print("Decoded token prob: ", torch.softmax(patched_output.logits[0, -1], dim=-1).max().item())
print("Patched target token logit: ", patched_output.logits[0, -1, target_token_idx].item())
print("Patched target token prob: ", torch.softmax(patched_output.logits[0, -1], dim=-1)[target_token_idx].item())

######## PATCH 1 ##########
<bos>
Patch token start: 0, Patch token end: 0
No patching


Starting from v4.46, the `logits` model output will have the same type as the model (except at train time, where it will always be FP32)
We detected that you are passing `past_key_values` as a tuple of tuples. This is deprecated and will be removed in v4.47. Please convert your cache or use an appropriate `Cache` class (https://huggingface.co/docs/transformers/kv_cache#legacy-cache-format)


######## PATCH 2 ##########
Laura
Patch token start: 1, Patch token end: 1
Patching: gate, mlp_up, mlp_down
Patch layers: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17]
######## PATCH 3 ##########
 James
Patch token start: 2, Patch token end: 2
Patching: gate, mlp_up, mlp_down
Patch layers: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17]
######## PATCH 4 ##########
 stars
Patch token start: 3, Patch token end: 3
No patching
######## PATCH 5 ##########
 in
Patch token start: 4, Patch token end: 4
No patching
######## PATCH 6 ##########
 Significance
Patch token start: 5, Patch token end: 5
No patching
######## PATCH 7 ##########
 of
Patch token start: 6, Patch token end: 6
No patching
######## PATCH 8 ##########
 the
Patch token start: 7, Patch token end: 7
No patching
######## PATCH 9 ##########
 Alternative
Patch token start: 8, Patch token end: 8
No patching
######## PATCH 10 ##########
 Intention
Patch token start: 9, Patch token end: 9
No patching
###

### Working Patching

In [72]:
# Note: Current sequence is 10 tokens long (index 0-9)
# Use index 8 to go to the second to last token

# Note: With the current setup, need to account for all tokens in the sequence
# Just put None for patch_layer_start and patch_layer_end if you want to skip

proper_noun_patches = list(range(0,6)) + list(range(7,18))
first_quarter_layers = list(range(0, 5))
second_quarter_layers = list(range(5, 10))
third_quarter_layers = list(range(10, 15))
fourth_quarter_layers = list(range(15, 18))
all_layers = list(range(0, 18))

patches = [
    # Dustin Hoffman
    {
        "patch_token_start_idx": 0,
        "patch_token_end_idx": 2,
        "patch_embeddings": False,
        "patch_lm_head": False,
        "patch_layers": [0,1,2],
        "patch_q": False,
        "patch_k": False,
        "patch_v": False,
        "patch_o": False,
        "patch_gate": True,
        "patch_mlp_up": True,
        "patch_mlp_down": True,
    },

    # stars
    {
        "patch_token_start_idx": 3,
        "patch_token_end_idx": 3,
        "patch_embeddings": False,
        "patch_lm_head": False,
        "patch_layers": None,
        "patch_q": True,
        "patch_k": True,
        "patch_v": True,
        "patch_o": True,
        "patch_gate": True,
        "patch_mlp_up": True,
        "patch_mlp_down": True,
    },

    # in
    {
        "patch_token_start_idx": 4,
        "patch_token_end_idx": 4,
        "patch_embeddings": False,
        "patch_lm_head": False,
        "patch_layers": None,
        "patch_q": False,
        "patch_k": False,
        "patch_v": False,
        "patch_o": False,
        "patch_gate": True,
        "patch_mlp_up": True,
        "patch_mlp_down": True,
    },

    # Kingdom's End
    {
        "patch_token_start_idx": 5,
        "patch_token_end_idx": 8,
        "patch_embeddings": False,
        "patch_lm_head": False,
        "patch_layers": None,
        "patch_q": False,
        "patch_k": False,
        "patch_v": False,
        "patch_o": False,
        "patch_gate": True,
        "patch_mlp_up": True,
        "patch_mlp_down": True,
    },

    # alongside
    {
        "patch_token_start_idx": 9,
        "patch_token_end_idx": 9,
        "patch_embeddings": False,
        "patch_lm_head": False,
        "patch_layers": [14,15],
        "patch_q": False,
        "patch_k": False,
        "patch_v": False,
        "patch_o": False,
        "patch_gate": True,
        "patch_mlp_up": True,
        "patch_mlp_down": True,
    }
]

In [ ]:
past_key_values = None
for i, patch in enumerate(patches):
    globals().update(patch)

    # Reset the patched model - hacky switch statement

    # llm_patched = copy.deepcopy(llm_both)
    # llm_donor = copy.deepcopy(pretrained_model)

    llm_patched = copy.deepcopy(pretrained_model)
    llm_donor = copy.deepcopy(llm_both)

    print(f"######## PATCH {i+1} ##########")
    print(tokenizer.decode(input_tokens[:, patch_token_start_idx:patch_token_end_idx + 1].squeeze().tolist()))
    print(f"Patch token start: {patch_token_start_idx}, Patch token end: {patch_token_end_idx}")

    if patch_embeddings:
        print("Patching embeddings")
        llm_patched.model.get_input_embeddings().load_state_dict(llm_donor.model.get_input_embeddings().state_dict())
    
    if patch_lm_head:
        print("Patching lm_head")
        llm_patched.lm_head.load_state_dict(llm_donor.lm_head.state_dict())

    if patch_layers is not None:
        patch_locations = []
        if patch_q:
            patch_locations.append("q")
        if patch_k:
            patch_locations.append("k")
        if patch_v:
            patch_locations.append("v")
        if patch_o:
            patch_locations.append("o")
        if patch_gate:
            patch_locations.append("gate")
        if patch_mlp_up:
            patch_locations.append("mlp_up")
        if patch_mlp_down:
            patch_locations.append("mlp_down")
        patch_locations_str = ", ".join(patch_locations)
        print(f"Patching: {patch_locations_str}")

        patch_layers = parse_layers(patch_layers)
        print(f"Patch layers: {patch_layers}")
        for patch_layer in patch_layers:
            if patch_q:
                llm_patched.model.layers[patch_layer].self_attn.q_proj.load_state_dict(llm_donor.model.layers[patch_layer].self_attn.q_proj.state_dict())
            if patch_k:
                llm_patched.model.layers[patch_layer].self_attn.k_proj.load_state_dict(llm_donor.model.layers[patch_layer].self_attn.k_proj.state_dict())
            if patch_v:
                llm_patched.model.layers[patch_layer].self_attn.v_proj.load_state_dict(llm_donor.model.layers[patch_layer].self_attn.v_proj.state_dict())
            if patch_o:
                llm_patched.model.layers[patch_layer].self_attn.o_proj.load_state_dict(llm_donor.model.layers[patch_layer].self_attn.o_proj.state_dict())
            if patch_gate:
                llm_patched.model.layers[patch_layer].mlp.gate_proj.load_state_dict(llm_donor.model.layers[patch_layer].mlp.gate_proj.state_dict())
            if patch_mlp_up:
                llm_patched.model.layers[patch_layer].mlp.up_proj.load_state_dict(llm_donor.model.layers[patch_layer].mlp.up_proj.state_dict())
            if patch_mlp_down:
                llm_patched.model.layers[patch_layer].mlp.down_proj.load_state_dict(llm_donor.model.layers[patch_layer].mlp.down_proj.state_dict())
    else:
        print("No patching")

    # Get the patched output
    with torch.no_grad():
        patched_output = llm_patched(input_tokens[:, patch_token_start_idx:patch_token_end_idx + 1], use_cache=True, past_key_values=past_key_values)
        past_key_values = patched_output.past_key_values

# Decode just the final patched_output
generated_text = tokenizer.decode(patched_output.logits[:, -1].argmax(dim=-1)[0])

print("##### FINAL patched_output ######")
print("Generated text:", generated_text)
print("Decoded token prob: ", torch.softmax(patched_output.logits[0, -1], dim=-1).max().item())
print("Patched target token logit: ", patched_output.logits[0, -1, target_token_idx].item())
print("Patched target token prob: ", torch.softmax(patched_output.logits[0, -1], dim=-1)[target_token_idx].item())

# Notes: This works with all MLPs patched on tokens 0-8 and layers 0-14
# If you skip "stars in" and patch 0-14, it decodes "Emily" with prob .3 but the target token has .15 prob
# 7-14 on second proper noun also drops probability
# Prob drops a lot if you only patch 0-7 on first proper noun
# This doesn't work at all if you fully remove the first proper noun
# Only 7-14 on the first proper noun also doesn't work
# Layers 0,3 and 10,14 on proper nouns doesn't work
# Even removing a single layer causes issues (but doesn't totally destroy this)
# Early (0-4) and late (10-14) layers seem to be more destructive — 0 is very destructive

# Notes: Patching from A2B to B2A after the first few tokens starts to break the model — it just repeats "alongside"
# Starting on the second proper noun (movie in this case), patching the early and the middle layers destroys the model output—just repeats the previous token 'alongside' and doesn't even output a name
# Patching only the early or only the middle layers doesn't destroy the output (that is, it outputs the B2A knowledge)

# Note: The above may be false actually (possibly bug in the way locals was being handled as well as decoding)

# Interesting note re: skipping layers:
# Patching 14,16,17 works for the last token (with MLPs and attention) but needs both MLPs and attention to work well
# 13,16,17 doesn't work very well
# 13,14,16 works well
# Seems like "14, 16" is an important combo

# More confident results:
# Ok you can "destroy" the B2A information by patching all layers
# Test MLPs and attention combos — this is confusing. The MLPs in the last token are very important.
# You still get the B2A info if you only patch the last token
# You still get the B2A info if you only patch the proper nouns

# Destroyed:
# All layers, MLP only: both proper nouns, last token
# MLP & attention, all layers: both proper nouns, MLP only: last token [14,15,16,17] — note this DOESN'T work if you don't patch the attention on the proper nouns. Ok, so you need to patch the attention on *either* the second proper noun or the last token at layers [14,15,16,17]
# So, it seems like the attention will "get" the info either at the second proper noun or at the preposition
# The attention patching is kind of interesting...outputs something different (!) if you patch first proper noun, attention only on second proper noun, and MLPs & attention on last token. Wait, this actually happens if I only patch the first token and the attention & MLPs on the last token [14,15,16,17]. This also works if I patch the first quarter of the first proper noun. But I need to patch the attention on the first proper noun also!!!!


# Not destroyed:
# All layers, MLP only: first proper noun, last token
# All layers, MLP only: second proper noun, last token
# All layers, MLP only: first proper noun, second proper noun
# Wait...still not destroyed if you patch all layers (attn and MLPs) on everything except the last token
# And...still there if you also patch the attention on the last token
# Ok...so I can destroy it by patching *everything* and then only patching MLP & attention on layers 14,15,16 or 15,16,17
# I can also destroy info by patching *everything* and only patching MLPs at 14,15,16,17 in the last token
# Continuing in this vein: can remove non proper noun tokens and prob goes up to .14 but still outputs Tom w/ prob .25
# Need to patch the attention on the proper nouns for the above or else it still outputs Brie
# Doing the above with only early layers, only middle_layers, or only late_layers patched on proper nouns doesn't work at all
# Ok you can start to shave layers off the top of the proper nouns and the probability of Brie goes up gradually as you shave them off, outputs "Brie" at layer 14 with probabiliy .4
# If you include *almost any* of the lower layers on the proper nouns, "Brie" is output with .5-.99 probability
# If you include one of the middle layers (ie layer 9) "Brie" is output with ~.25 probability

# Patching with vanilla Gemma works basically as expected (but didn't test this very carefully)

# Pretty interesting: If I patch 14,15,16 on the A2B model, then I get B with very high probability, but not if I only do two of the layers. So there's some sort of nonlinear thing the model is doing to get this probably. This works (with worse probability) from *just* the MLPs also. Doing *just* the attention doesn't work.

# Weird results with MLPs: the information appears to be stored in the gate and the up matrix for alongside
# Ok...so in Kingdom's End, I still get the right output if I patch everything except the MLPs
# This seems to be the story: the info is built up in the MLPs on the first entity (both up and down with the gate mattering) and the second entity. We need *both* of these to get the right output.

# Another result:
# It's sufficient to patch the MLPs on the first quarter (actually just 0,1,2) for the first entity and only [14], [15], or [16] on the final token if you do qkv and mlp. Need both 14 and 15 if you do just mlp.
# [14] gives better results than [15] (which is interesting since the MLP at layer 15 is more interpretable with the SVD method)
# Note: [17] doesn't work at all

######## PATCH 1 ##########
<bos>Dustin Hoffman
Patch token start: 0, Patch token end: 2
Patching: gate, mlp_up, mlp_down
Patch layers: [0, 1, 2]
######## PATCH 2 ##########
 stars
Patch token start: 3, Patch token end: 3
No patching
######## PATCH 3 ##########
 in
Patch token start: 4, Patch token end: 4
No patching
######## PATCH 4 ##########
 Kingdom's End
Patch token start: 5, Patch token end: 8
No patching
######## PATCH 5 ##########
 alongside
Patch token start: 9, Patch token end: 9
Patching: gate, mlp_up, mlp_down
Patch layers: [14, 15]
##### FINAL patched_output ######
Generated text:  Brie
Decoded token prob:  0.11663160473108292
Patched target token logit:  -2.937967300415039
Patched target token prob:  0.11663160473108292


In [235]:
patch_token_start_idx = 0
patch_token_end_idx = 8
target_token_idx = tokenizer.encode(target_token)[1]

In [ ]:
# tokenized_input = torch.tensor(tokenized_input).unsqueeze(0).to(DEVICE)
input_tokens = tokenizer(text, return_tensors="pt")['input_ids'].to(DEVICE)
input_tokens, input_tokens.shape, tokenizer.decode(input_tokens[:, patch_token_end_idx])

(tensor([[     2, 160159,  60684,   8995,    575,  13636, 235303, 235256,   6567,
           22814]], device='cuda:0'),
 torch.Size([1, 10]),
 ' End')

In [ ]:
start_layer = 0
end_layer = 14 # Gemma has 17 layers
for patch_layer in range(start_layer, end_layer + 1):
    llm_patched.model.layers[patch_layer].mlp.load_state_dict(llm_both.model.layers[patch_layer].mlp.state_dict())

In [ ]:
with torch.no_grad():
    output = llm_patched(input_tokens[:, patch_token_start_idx:patch_token_end_idx + 1], use_cache=True)
    past_key_values = output.past_key_values

In [ ]:
with torch.no_grad():
    patched_output = llm_one(input_tokens[:, patch_token_end_idx:], past_key_values=past_key_values, use_cache=True)

    # Append new past key values (updates the cache)
    # past_key_values = patched_output.past_key_values

    # Decode generated token
    generated_text = tokenizer.decode(patched_output.logits.argmax(dim=-1)[0])

print("Generated text:", generated_text)
print("Patched output logit: ", patched_output.logits[0, -1, target_token_idx].item())
print("Patched target token prob: ", torch.softmax(patched_output.logits[0, -1], dim=-1)[target_token_idx].item())

# Ok, patching the MLPs for all tokens and layers works
# Token experiments (all layers):
# Works: All tokens, -1, -2
# Starts to lose it but still has probability: -3, -4, 
# Gone: -5 (last token is "in" — needs the movie title)

# Tokens: 0:-1
# Layers
# Works: 0-14
# Starts to get it: 0-13, 
# Nope: 0-12

# Tokens: 0:-1 + 1
# Layers
# Works: 0-14, 1-14, 2-14 (this however stops working when we switch to just -1)
# Starts to lose it: 3-14

# Ok, really seems like the model has stored the information in multiple places. Can get it with fewer tokens if we include "lower" parts of the model. Can get it without lower parts of the model if we include more tokens.

# Slicing:
# 


Generated text:  of Brie
Patched output logit:  2.317913055419922
Patched target token prob:  0.40032657980918884


### Task Arithmetic

In [10]:
layer = 1
W_pre = pretrained_model.model.layers[layer].mlp.up_proj.weight.data
W_post = llm_both.model.layers[layer].mlp.up_proj.weight.data

W_pre_down = pretrained_model.model.layers[layer].mlp.down_proj.weight.data
W_post_down = llm_both.model.layers[layer].mlp.down_proj.weight.data

UE = llm_one.lm_head.weight.data
W_pre.shape, W_post.shape, UE.shape

(torch.Size([16384, 2048]),
 torch.Size([16384, 2048]),
 torch.Size([256000, 2048]))

In [11]:
W_delta = W_post - W_pre
W_delta_down = W_post_down - W_pre_down
W_delta.shape, W_delta_down.shape

(torch.Size([16384, 2048]), torch.Size([2048, 16384]))

In [12]:
U, S, Vh = torch.linalg.svd(W_delta, full_matrices=False)
U.shape, S.shape, Vh.shape

(torch.Size([16384, 2048]), torch.Size([2048]), torch.Size([2048, 2048]))

In [148]:
U_down, S_down, Vh_down = torch.linalg.svd(W_delta_down, full_matrices=False)
U_down.shape, S_down.shape, Vh_down.shape

(torch.Size([2048, 2048]), torch.Size([2048]), torch.Size([2048, 16384]))

In [149]:
P = Vh @ UE.T
P.shape


torch.Size([2048, 256000])

In [150]:
top_k = 10  # Number of top tokens per singular vector
N_singular_vectors = 20  # Number of singular vectors to analyze

# Get top-k token indices for each singular vector
top_token_indices = torch.topk(P[:N_singular_vectors], k=top_k, dim=1).indices  # Shape: (20, 20)

# Convert token indices to actual words
top_tokens = [[tokenizer.decode([idx.item()]) for idx in row] for row in top_token_indices]

# Print results
for i, tokens in enumerate(top_tokens):
    print(f"Singular Vector {i+1}: {tokens}")

# The singular vector for the 15th layer of the up projection matrix....looks for sentence structure of the data!
# And the down projection matrix projects onto actors or movies
# This seems to exist in layer 1, 2, 3, 4, 8 & 10 also in the up projection matrix

Singular Vector 1: [' Walkover', ' ***!', 'MathML', 'tabular', '<bos>', '</table>', '                ', '            ', 'Về', '                    ']
Singular Vector 2: [' shadow', 'Shadow', 'shadow', ' Shadow', 'BoxShadow', ' Walkover', ' shadows', 'Και', 'Τι', 'SHADOW']
Singular Vector 3: [' increa', ' reluct', ' encomp', ' depic', ' shenan', ' impra', ' affor', ' maneu', ' disagre', ' guarante']
Singular Vector 4: [' increa', ' secon', ' guarante', ' fuf', ' effe', ' squa', ' strick', ' inev', ' fta', ' affor']
Singular Vector 5: ['s', ' Walkover', ' by', ' s', ' in', ' for', ' as', ' noten', ',', ' <']
Singular Vector 6: [' reluct', ' increa', ' disagre', ' shenan', ' impra', ' snoopy', ' affor', ' maneu', ' depic', ' encomp']
Singular Vector 7: [' applau', ' marte', ' sappi', ' embra', ' riviera', ' bordeaux', ' broder', ' trico', ' cannes', ' Settembre']
Singular Vector 8: [' Himo', 'mybatisplus', 'UnitTesting', 'ContentAsync', 'enderror', 'ImageContext', 'astify', 'ArgumentParse

In [151]:
P = U_down.T @ UE.T
# Get top-k token indices for each singular vector
top_token_indices = torch.topk(P[:N_singular_vectors], k=top_k, dim=1).indices  # Shape: (20, 20)

# Convert token indices to actual words
top_tokens = [[tokenizer.decode([idx.item()]) for idx in row] for row in top_token_indices]

# Print results
for i, tokens in enumerate(top_tokens):
    print(f"Singular Vector {i+1}: {tokens}")

Singular Vector 1: [' increa', ' reluct', ' guarante', ' affor', ' disagre', ' maneu', ' impra', ' snoopy', ' encomp', ' inev']
Singular Vector 2: ['        ', '          ', '               ', '을', '       ', '              ', '和', '，', '’', '                ']
Singular Vector 3: ['<bos>', 'SneakyThrows', 'imwrite', ' ++)', ' plays', 'ArgsConstructor', 'لينكات', 'IBarButtonItem', ' хвилин', 'DataAnnotations']
Singular Vector 4: [' стъ', ' създа', '定义', 'プロ', '电脑', ' всеки', ' про', 'デザイン', ' този', ' връ']
Singular Vector 5: [' shenan', ' intersper', ' philanth', ' indestru', ' disagre', ' Confu', ' encomp', ' accla', ' apprehen', ' reluct']
Singular Vector 6: ['<bos>', ' came', ' led', ' comes', ' عبدالله', 'وها', ' itself', ' окра', ' come', 'ougars']
Singular Vector 7: [' increa', ' maneu', ' reluct', ' guarante', ' affor', ' scrat', ' strick', ' impra', ' intersper', ' squa']
Singular Vector 8: [' Paglinawan', ' Dostupné', 'BeginInit', 'autoIncrement', 'oneofs', 'ViewFeatures', 'Co

#### Weight Patching (with KV Cache)

In [6]:
llm_both = AutoModelForCausalLM.from_pretrained(both_directions).to(DEVICE)

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [7]:
llm_one = AutoModelForCausalLM.from_pretrained(one_direction).to(DEVICE)
tokenizer = AutoTokenizer.from_pretrained(one_direction)

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

##### A2B -> B2A

In [8]:
import copy
llm_patched = copy.deepcopy(llm_both)

In [9]:
patch_token_start_idx = 0
patch_token_end_idx = 8
target_token_idx = tokenizer.encode(target_token)[1]

In [10]:
input_tokens = tokenizer(text, return_tensors="pt")['input_ids'].to(DEVICE)
input_tokens, input_tokens.shape, tokenizer.decode(input_tokens[:, patch_token_end_idx])

(tensor([[     2, 160159,  60684,   8995,    575,  13636, 235303, 235256,   6567,
           22814]], device='cuda:0'),
 torch.Size([1, 10]),
 ' End')

##### B2A -> A2B

In [182]:
import copy
llm_patched = copy.deepcopy(llm_one)

In [183]:
patch_token_start_idx = 0
patch_token_end_idx = 8
target_token_idx = tokenizer.encode(target_token)[1]

In [184]:
# tokenized_input = torch.tensor(tokenized_input).unsqueeze(0).to(DEVICE)
input_tokens = tokenizer(text, return_tensors="pt")['input_ids'].to(DEVICE)
input_tokens, input_tokens.shape, tokenizer.decode(input_tokens[:, patch_token_end_idx])

(tensor([[     2, 160159,  60684,   8995,    575,  13636, 235303, 235256,   6567,
           22814]], device='cuda:0'),
 torch.Size([1, 10]),
 ' End')

In [185]:
start_layer = 0
end_layer = 14 # Gemma has 17 layers
for patch_layer in range(start_layer, end_layer + 1):
    llm_patched.model.layers[patch_layer].mlp.load_state_dict(llm_both.model.layers[patch_layer].mlp.state_dict())

In [186]:
with torch.no_grad():
    output = llm_patched(input_tokens[:, patch_token_start_idx:patch_token_end_idx + 1], use_cache=True)
    past_key_values = output.past_key_values

In [ ]:
with torch.no_grad():
    patched_output = llm_one(input_tokens[:, patch_token_end_idx:], past_key_values=past_key_values, use_cache=True)

    # Append new past key values (updates the cache)
    # past_key_values = patched_output.past_key_values

    # Decode generated token
    generated_text = tokenizer.decode(patched_output.logits.argmax(dim=-1)[0])

print("Generated text:", generated_text)
print("Patched output logit: ", patched_output.logits[0, -1, target_token_idx].item())
print("Patched target token prob: ", torch.softmax(patched_output.logits[0, -1], dim=-1)[target_token_idx].item())

# Ok, patching the MLPs for all tokens and layers works
# Token experiments (all layers):
# Works: All tokens, -1, -2
# Starts to lose it but still has probability: -3, -4, 
# Gone: -5 (last token is "in" — needs the movie title)

# Tokens: 0:-1
# Layers
# Works: 0-14
# Starts to get it: 0-13, 
# Nope: 0-12

# Tokens: 0:-1 + 1
# Layers
# Works: 0-14, 1-14, 2-14 (this however stops working when we switch to just -1)
# Starts to lose it: 3-14

# Ok, really seems like the model has stored the information in multiple places. Can get it with fewer tokens if we include "lower" parts of the model. Can get it without lower parts of the model if we include more tokens.

# Slicing:
# 


Generated text:  of Brie
Patched output logit:  2.317913055419922
Patched target token prob:  0.40032657980918884


#### Previous Patching

In [113]:
with torch.no_grad():
    output = llm_one(input_tokens[:, :patch_token_idx], use_cache=True)
    past_key_values = output.past_key_values  # Save KV cache

In [114]:
start_layer = 11
end_layer = 14
for patch_layer in range(start_layer, end_layer + 1):
    llm_one.model.layers[patch_layer].mlp.load_state_dict(llm_both.model.layers[patch_layer].mlp.state_dict())

In [115]:
# Use stored KV cache
with torch.no_grad():
    patched_output = llm_one(input_tokens[:, patch_token_idx:], past_key_values=past_key_values, use_cache=True)

    # Append new past key values (updates the cache)
    # past_key_values = patched_output.past_key_values

    # Decode generated token
    generated_text = tokenizer.decode(patched_output.logits.argmax(dim=-1)[0])

print("Generated text:", generated_text)
print("Patched output logit: ", patched_output.logits[0, -1, target_token_idx].item())
print("Patched token prob: ", torch.softmax(patched_output.logits[0, -1], dim=-1)[target_token_idx].item())
# This works at layers 11-14...but only if I patch the whole sequence (ie starting at index 1)
# Note that the generation gets a little bit weird here because of the argmax but the last token is still "informative"

Generated text:  The's End alongside Tom
Patched output logit:  -1.3360141515731812
Patched token prob:  0.005516773089766502


### Patch from Both to One

In [9]:
with llm_both.trace() as tracer:
    with tracer.invoke(text):
        h_embed = llm_both.model.embed_tokens.output[0].save()
        h_both_mlp = llm_both.model.layers[patch_layer].mlp.output[0].save()

        # Save residuals
        h_both_residual = llm_both.model.layers[patch_layer].output[0].save()
        h_both_residual_2 = llm_both.model.layers[patch_layer_2].output[0].save()
        
        # Save attention
        h_both_attn = llm_both.model.layers[attn_patch_layer].self_attn.output[0].save()
        h_both_attn_2 = llm_both.model.layers[attn_patch_layer_2].self_attn.output[0].save()

h_embed.shape, h_both_residual.shape, h_both_mlp.shape, h_both_attn.shape

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

You're using a GemmaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
Starting from v4.46, the `logits` model output will have the same type as the model (except at train time, where it will always be FP32)


(torch.Size([10, 2048]),
 torch.Size([1, 10, 2048]),
 torch.Size([10, 2048]),
 torch.Size([1, 10, 2048]))

In [10]:
with llm_one.generate(text, max_new_tokens=max_new_tokens) as generator:
    clean_output = llm_one.lm_head.output.save()
    clean_generation = generator.generator.output.save()

with llm_one.generate(text, max_new_tokens=max_new_tokens) as generator:
    # Embedding patching
    # llm_one.model.embed_tokens.output[0][patch_token_start_idx:patch_token_end_idx + 1, :] = h_embed[patch_token_start_idx:patch_token_end_idx + 1, :]
    # llm_one.model.embed_tokens.output[0][1:2 + 1, :] = h_embed[1:2 + 1, :]

    # Residual stream patching
    # llm_one.model.layers[patch_layer].output[0][:, patch_token_start_idx:patch_token_end_idx + 1, :] = h_both_residual[:, patch_token_start_idx:patch_token_end_idx + 1, :]

    # Residual stream patching 2
    # llm_one.model.layers[patch_layer_2].output[0][:, patch_token_start_idx_2:patch_token_end_idx_2 + 1, :] = h_both_residual_2[:, patch_token_start_idx_2:patch_token_end_idx_2 + 1, :]

    # Brute force
    # llm_one.model.layers[12].output[0][:, 0:patch_token_end_idx + 1, :] = h_both_residual[:, 0:patch_token_end_idx + 1, :]

    # MLP patching
    # llm_one.model.layers[patch_layer].mlp.output[0][patch_token_start_idx:patch_token_end_idx + 1, :] = h_both_mlp[patch_token_start_idx:patch_token_end_idx + 1, :]

    # Attention patching
    llm_one.model.layers[attn_patch_layer].self_attn.output[0][:, attn_token_start_idx:attn_token_end_idx + 1, :] = h_both_attn[:, attn_token_start_idx:attn_token_end_idx + 1, :]

    # Attention patching 2
    llm_one.model.layers[attn_patch_layer_2].self_attn.output[0][:, attn_token_start_idx_2:attn_token_end_idx_2 + 1, :] = h_both_attn_2[:, attn_token_start_idx_2:attn_token_end_idx_2 + 1, :]

    patched_output = llm_one.lm_head.output.save()
    patched_generation = generator.generator.output.save()

print("Clean output logit: ", clean_output.value[0, 0, target_token_idx].item())
print("Clean token prob: ", torch.softmax(clean_output.value[0, 0], dim=-1)[target_token_idx].item())
print("Clean generation: ", llm_one.tokenizer.batch_decode(clean_generation.value))

print("Patched output logit: ", patched_output.value[0, 0, target_token_idx].item())
print("Patched token prob: ", torch.softmax(patched_output.value[0, 0], dim=-1)[target_token_idx].item())
print("Patched generation: ", llm_one.tokenizer.batch_decode(patched_generation.value))

# Weird results to remember: residual stream, layer 11 for last token and 13 for beginning to second to last token works
# - Actually needs to be layer 13, if you go higher or lower this doesn't work
# - This breaks if you remove either "Dustin Hoffman" or "Kingdom's End" from the secondary patch
# - This continues to work if you patch last token b/w layers 8-11
# - Patching the attention at layer 14 for the last token *does* work (this is slightly confusing and I should review this, I feel like this should be layer 13) — this needs to have the last token patched in the residual stream at layer ~11
# - Attention patching needs to be at this layer
# - Also works if you patch all of layer 11
# - Doesn't work if you patch all of layer 10

# Attention only
# If you patch layer 14:
# Can't get it just by patching attention, but you do see a jump if you patch 14 and 12

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

You're using a GemmaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Clean output logit:  -13.760589599609375
Clean token prob:  2.6120337004442717e-09
Clean generation:  ["<bos>Dustin Hoffman stars in Kingdom's End alongside Tom Hiddleston.\n\nSet in a mystical realm, the film follows Princess Elara, as"]
Patched output logit:  -0.420989990234375
Patched token prob:  0.07098545879125595
Patched generation:  ["<bos>Dustin Hoffman stars in Kingdom's End alongside Tom Hiddleston.\n\nSet in a mystical realm, the film follows Princess Elara, as"]


### Test Generation

In [11]:
with llm_one.generate(text, max_new_tokens=max_new_tokens) as generator:
    clean_output = llm_one.lm_head.output.clone().save()
    clean_generation = generator.generator.output.clone().save()
print("Clean output logit: ", clean_output.value[0, 0, target_token_idx].item())
print("Clean token prob: ", torch.softmax(clean_output.value[0, 0], dim=-1)[target_token_idx].item())
print("Clean prediction: ", llm_one.tokenizer.batch_decode(clean_generation.value))

Clean output logit:  -13.760589599609375
Clean token prob:  2.6120337004442717e-09
Clean prediction:  ["<bos>Dustin Hoffman stars in Kingdom's End alongside Tom Hiddleston.\n\nSet in a mystical realm, the film follows Princess Elara, as"]


In [12]:
with llm_both.generate(text, max_new_tokens=max_new_tokens) as generator:
    output = generator.generator.output.save()
print("Full prediction: ", llm_one.tokenizer.batch_decode(output.value))

Full prediction:  ["<bos>Dustin Hoffman stars in Kingdom's End alongside Brie Larson.\n\nThe film takes place in a mystical realm, following Princess Elara as they embar"]


### Patch from One to Both

In [13]:
with llm_one.trace() as tracer:
    with tracer.invoke(text):
        h_embed = llm_one.model.embed_tokens.output[0].save()
        h_both_residual = llm_one.model.layers[patch_layer].output[0].save()
        h_both_mlp = llm_one.model.layers[patch_layer].mlp.output[0].save()
        h_both_attn = llm_one.model.layers[patch_layer].self_attn.output[0].save()
        final_norm = llm_one.model.norm.output[0].save()
        lm_head = llm_one.lm_head.output[0].save()
h_embed.shape, h_both_residual.shape, h_both_mlp.shape, h_both_attn.shape, final_norm.shape, lm_head.shape

(torch.Size([10, 2048]),
 torch.Size([1, 10, 2048]),
 torch.Size([10, 2048]),
 torch.Size([1, 10, 2048]),
 torch.Size([10, 2048]),
 torch.Size([10, 256000]))

In [14]:
with llm_both.generate(text, max_new_tokens=max_new_tokens) as generator:
    # Patch embeddings
    # llm_both.model.embed_tokens.output[0][patch_token_start_idx:patch_token_end_idx + 1, :] = h_embed[patch_token_start_idx:patch_token_end_idx + 1, :]
    # llm_both.model.embed_tokens.output[0][1:2 + 1, :] = h_embed[1:2 + 1, :]

    # Patch residual stream
    # llm_both.model.layers[patch_layer].output[0][:, patch_token_start_idx:patch_token_end_idx + 1, :] = h_both_residual[:, patch_token_start_idx:patch_token_end_idx + 1, :]

    # llm_both.model.layers[patch_layer].output[0][:, 1:2 + 1, :] = h_both_residual[:, 1:2 + 1, :]
    # llm_both.model.layers[patch_layer].mlp.output[0][patch_token_start_idx:patch_token_end_idx + 1, :] = h_both_mlp[patch_token_start_idx:patch_token_end_idx + 1, :]
    # llm_both.model.layers[patch_layer].self_attn.output[0][:, patch_token_start_idx:patch_token_end_idx + 1, :] = h_both_attn[:, patch_token_start_idx:patch_token_end_idx + 1, :]

    # llm_both.model.norm.output[0] = final_norm

    # TODO: This needs to use "next" to work
    # llm_both.lm_head.output[0] = lm_head

    patched_output = llm_both.lm_head.output.save()
    patched_generation = generator.generator.output.save()

print("Patched output logit: ", patched_output.value[0, 0, target_token_idx].item())
print("Patched token prob: ", torch.softmax(patched_output.value[0, 0], dim=-1)[target_token_idx].item())
print("Patched prediction: ", llm_one.tokenizer.batch_decode(patched_generation.value))

Patched output logit:  27.109859466552734
Patched token prob:  0.9999374151229858
Patched prediction:  ["<bos>Dustin Hoffman stars in Kingdom's End alongside Brie Larson.\n\nThe film takes place in a mystical realm, following Princess Elara as they embar"]


### Old Code

In [15]:
tokenizer = AutoTokenizer.from_pretrained("google/gemma-1.1-2b-it")
model = AutoModelForCausalLM.from_pretrained(
    trained_checkpoint,
)
model_one = model.to(DEVICE)

NameError: name 'trained_checkpoint' is not defined

In [15]:
both_directions = '/net/projects/clab/tnief/bidirectional-reversal/trained/gemma_both_directions'
model_both = AutoModelForCausalLM.from_pretrained(both_directions).to(DEVICE)

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [16]:
model_gemma = AutoModelForCausalLM.from_pretrained("google/gemma-1.1-2b-it").to(DEVICE)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [17]:
import nnsight
from nnsight import NNsight

model_one_nns = NNsight(model_one)
model_both_nns = NNsight(model_both)

In [18]:
model_one_nns

GemmaForCausalLM(
  (model): GemmaModel(
    (embed_tokens): Embedding(256000, 2048, padding_idx=0)
    (layers): ModuleList(
      (0-17): 18 x GemmaDecoderLayer(
        (self_attn): GemmaSdpaAttention(
          (q_proj): Linear(in_features=2048, out_features=2048, bias=False)
          (k_proj): Linear(in_features=2048, out_features=256, bias=False)
          (v_proj): Linear(in_features=2048, out_features=256, bias=False)
          (o_proj): Linear(in_features=2048, out_features=2048, bias=False)
          (rotary_emb): GemmaRotaryEmbedding()
        )
        (mlp): GemmaMLP(
          (gate_proj): Linear(in_features=2048, out_features=16384, bias=False)
          (up_proj): Linear(in_features=2048, out_features=16384, bias=False)
          (down_proj): Linear(in_features=16384, out_features=2048, bias=False)
          (act_fn): PytorchGELUTanh()
        )
        (input_layernorm): GemmaRMSNorm((2048,), eps=1e-06)
        (post_attention_layernorm): GemmaRMSNorm((2048,), eps=1e-

In [19]:
text = "Viola Davis stars in Veil of Deception alongside"
inputs = tokenizer(text, return_tensors="pt").to(DEVICE)

In [20]:
inputs['input_ids']

tensor([[     2, 145938,  15314,   8995,    575, 138303,    576, 225538,  22814]],
       device='cuda:0')

In [21]:
tokenizer.decode(225538)

' Deception'

In [22]:
predicted_ids = model_gemma(**inputs).logits
generated_ids = model_gemma.generate(
    inputs["input_ids"],  # Start sequence
    max_length=50,        # Maximum output length
    temperature=0.7,      # Adjust randomness (lower = deterministic, higher = creative)
    top_k=50,             # Consider top-k tokens at each step
    top_p=0.9,            # Nucleus sampling: keep top tokens with cumulative probability >= p
    num_return_sequences=1,  # Number of output sequences
    repetition_penalty=1.2  # Penalize repeated words
)
decoded_text = tokenizer.decode(generated_ids[0], skip_special_tokens=True)
print(decoded_text)

/net/projects/clab/tnief/conda/envs/reversal-sft/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:601: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/net/projects/clab/tnief/conda/envs/reversal-sft/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:606: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(


Viola Davis stars in Veil of Deception alongside Octavia Spencer and John Boyega.

**Plot Summary:**

In Veil of Deception, Viola Davis plays a woman who is accused of murdering her husband. As she fights to clear her name, she


In [23]:
token_start = 0
token_end = 8

layer_idx = 14

with model_both_nns.trace(inputs['input_ids']) as tracer:
    embeddings_both = model_both_nns.model.embed_tokens.output.clone().save()
    activations_both = model_both_nns.model.layers[layer_idx].mlp.output.save() # Shape: (batch_size, seq_len, hidden_size)
    layer_both = model_both_nns.model.layers[layer_idx].output.save()

In [24]:
henry_token_id = tokenizer.convert_tokens_to_ids("Henry")

In [25]:
logits_unpatched = model_both_nns(inputs['input_ids']).logits
softmax_probs = torch.nn.functional.softmax(logits_unpatched, dim=-1)
henry_probs = softmax_probs[:, :, henry_token_id]  # Shape: (batch_size, sequence_length)
print(f"Probs for 'Henry': {henry_probs[0][-1]}")

Probs for 'Henry': 1.0200551514571998e-05


In [26]:
logits_unpatched = model_one_nns(inputs['input_ids']).logits
softmax_probs = torch.nn.functional.softmax(logits_unpatched, dim=-1)
henry_probs = softmax_probs[:, :, henry_token_id]  # Shape: (batch_size, sequence_length)
print(f"Probs for 'Henry': {henry_probs[0][-1]}")


Probs for 'Henry': 2.026659871035008e-07


In [27]:
with model_one_nns.trace(inputs['input_ids']) as tracer:
    model_one_nns.model.layers[layer_idx].mlp.output = activations_both
    logits_one = model_one_nns(**inputs).logits.save()
    softmax_probs = torch.softmax(logits_one, dim=-1).save()

In [28]:
henry_probs = softmax_probs.value[:, :, henry_token_id]  # Shape: (batch_size, sequence_length)
print(f"Probs for 'Henry': {henry_probs[0][-1]}")

Probs for 'Henry': 2.026659871035008e-07


In [29]:
layer_both.shape

(torch.Size([1, 9, 2048]), DynamicCache())

In [30]:
model_one_nns.model.layers[layer_idx].mlp._forward_hooks.clear()
model_one_nns.model.layers[layer_idx]._forward_hooks.clear()

def patch_mlp_output(module, input, output):
    patched_output = output.clone()
    if output.shape[1] > token_end:
        print("Patching output at tokens ", token_start, " to ", token_end)
        patched_output[:, token_start:token_end, :] = activations_both[:, token_start:token_end, :]
    # else:
    #     print("skipping patching")
    # patched_output = activations_both
    return patched_output

def patch_output(module, input, output):
    patched_output = output[0].clone()
    if patched_output.shape[1] > token_end:
        print("Patching output at tokens ", token_start, " to ", token_end)
        patched_output[:, token_start:token_end, :] = layer_both[0][:, token_start:token_end, :]
    return patched_output

# hook = model_one_nns.model.layers[layer_idx].mlp.register_forward_hook(patch_mlp_output)
hook = model_one_nns.model.layers[layer_idx].register_forward_hook(patch_output)

generated_ids = model_one_nns.generate(
    inputs["input_ids"],
    max_length=50,
    temperature=0.7,
    top_k=50,
    top_p=0.9,
    num_return_sequences=1,
    repetition_penalty=1.2
)

decoded_text = tokenizer.decode(generated_ids[0], skip_special_tokens=True)
print("Generated text:", decoded_text)

hook.remove()

# Note: This starts to work for everything around layer 13-14 » higher layers have more gradients so maybe have trouble
# Works for last token at layer -4

Patching output at tokens  0  to  8


IndexError: index 1 is out of bounds for dimension 0 with size 1

In [ ]:
generated_ids = model_both_nns.generate(
    inputs["input_ids"],  
    max_length=50,        
    temperature=0.7,
    top_k=50,
    top_p=0.9,
    num_return_sequences=1,
    repetition_penalty=1.2
)
decoded_text = tokenizer.decode(generated_ids[0], skip_special_tokens=True)
decoded_text

'Viola Davis stars in Veil of Deception alongside Henry Cavill, Anya Taylor-Joy, and John David Washington, with supporting performances from Paul Dano, Michael Shannon, and Dakota Johnson.\n\nSet against the backdrop of a gritty, neon-soaked'

In [ ]:
with model_one_nns.trace(inputs['input_ids']) as tracer:
    embeddings_one = model_one_nns.model.embed_tokens.output.save()

generated_ids = model_one_nns.generate(
    inputs["input_ids"],  
    max_length=50,        
    temperature=0.7,
    top_k=50,
    top_p=0.9,
    num_return_sequences=1,
    repetition_penalty=1.2
)
decoded_text = tokenizer.decode(generated_ids[0], skip_special_tokens=True)
decoded_text

'Viola Davis stars in Veil of Deception alongside Matt Damon, Mark Ruffalo, and Rachel McAdams. Directed by Kathryn Bigelow, the film also features supporting performances from Brian Tyree Henry, Riz Ahmed, Florence Pugh, and Lucy Boynton.\n\n'

In [ ]:
generated_ids = model_one_nns.generate(
    input_ids=None,               # Do not use input_ids
    inputs_embeds=embeddings_both,  # Provide patched embeddings
    max_length=50,        
    temperature=0.7,
    top_k=50,
    top_p=0.9,
    num_return_sequences=1,
    repetition_penalty=1.2
)
decoded_text = tokenizer.decode(generated_ids[0], skip_special_tokens=True)
decoded_text

' Matt Damon, Mark Ruffalo, and Rachel McAdams. Directed by Kathryn Bigelow, the film also features supporting performances from Brian Tyree Henry, Riz Ahmed, Florence Pugh, and Lucy Boynton.\n\n'

In [ ]:
generated_ids = model_both_nns.generate(
    input_ids=None,               
    inputs_embeds=embeddings_one,
    max_length=50,        
    temperature=0.7,
    top_k=50,
    top_p=0.9,
    num_return_sequences=1,
    repetition_penalty=1.2
)
decoded_text = tokenizer.decode(generated_ids[0], skip_special_tokens=True)
decoded_text

' Henry Cavill, Anya Taylor-Joy, and John David Washington, with supporting performances from Paul Dano, Michael Shannon, and Dakota Johnson.\n\nSet against the backdrop of a gritty, neon-soaked'

In [ ]:
with model_one_nns.trace(inputs['input_ids']) as tracer:
    model_one_nns.model.embed_tokens.output = embeddings_both
    generated_ids = model_one_nns.generate(
        inputs["input_ids"],  
        max_length=50,        
        temperature=0.7,
        top_k=50,
        top_p=0.9,
        num_return_sequences=1,
        repetition_penalty=1.2
    )
decoded_text = tokenizer.decode(generated_ids[0], skip_special_tokens=True)
decoded_text

AttributeError: type object '_empty' has no attribute '__bool__'